In [38]:
import pandas as pd
import numpy as np

In [39]:
data = pd.read_csv("pii.csv", sep='|')

In [40]:
data['vb_tsmart_zip'] = data.vb_tsmart_zip.apply(lambda x: '0' + str(x) if len(str(x)) != 5 else x)

In [41]:
data['vb_tsmart_zip'] = data['vb_tsmart_zip'].astype(str)

In [42]:
data[data.vb_tsmart_zip.apply(lambda x: True if len(str(x)) < 4 else False)].vb_tsmart_zip

1791      00
1842      00
2458      00
6281      00
6757      00
          ..
983942    00
984618    00
986565    00
993671    00
994928    00
Name: vb_tsmart_zip, Length: 674, dtype: object

In [24]:
data.to_csv("pii_zipcode_fixed.csv")

In [58]:
data = data.reset_index()
data.columns

Index(['index', 'vb_email', 'vb_tsmart_first_name', 'vb_tsmart_middle_name',
       'vb_tsmart_last_name', 'vb_voterbase_phone',
       'vb_voterbase_phone_wireless', 'vb_voterbase_dob',
       'coalesced_noncommercial_age', 'gender_noncommercial',
       'vb_tsmart_exact_track', 'vb_tsmart_full_address', 'vb_tsmart_city',
       'vb_tsmart_state', 'vb_tsmart_zip', 'vb_tsmart_zip4',
       'vb_tsmart_street_number', 'vb_tsmart_street_name',
       'vb_tsmart_street_suffix', 'vb_vf_reg_cass_address_full',
       'vb_vf_reg_cass_city', 'vb_vf_reg_cass_state', 'vb_vf_reg_cass_zip',
       'vb_vf_reg_cass_zip4', 'vb_vf_reg_cass_street_num',
       'vb_vf_reg_cass_street_name', 'vb_vf_reg_cass_street_suffix',
       'ts_reg_urbanicity_bins', 'full_name'],
      dtype='object')

In [59]:
data['full_name'] = data.vb_tsmart_first_name + ' ' + data.vb_tsmart_last_name

In [69]:
f1_both_email_phone = data[(data.vb_voterbase_phone.notnull() | data.vb_voterbase_phone_wireless.notnull()
                           ) & (data.vb_email.notnull())]

In [70]:
f1_both_email_phone.shape

(328524, 29)

In [71]:
subset1 = data[~data.index.isin(f1_both_email_phone.index)]
f2_phone_only = subset1[(subset1.vb_voterbase_phone.notnull()) | (data.vb_voterbase_phone_wireless.notnull()) 
                        & (subset1.vb_email.isnull())]
f2_phone_only.shape

/Users/gabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


(325907, 29)

In [72]:
subset1.shape

(671476, 29)

In [76]:
subset2 = data[(~data.index.isin(f1_both_email_phone.index)) 
               & (~data.index.isin(f2_phone_only.index))]
f3_email_only = subset2[(subset2.vb_email.notnull()) & ((subset1.vb_voterbase_phone.isnull()) & 
                        (data.vb_voterbase_phone_wireless.isnull()))]
f3_email_only.shape

/Users/gabriel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


(57875, 29)

In [77]:
subset3 = data[(~data.index.isin(f1_both_email_phone.index)) 
               & (~data.index.isin(f2_phone_only.index))
              & (~data.index.isin(f3_email_only.index))]

In [78]:
f4_both_missing = subset3[(subset3.vb_email.isnull()) & (
    subset3.vb_voterbase_phone.isnull() & subset3.vb_voterbase_phone_wireless.isnull()

)]

In [79]:
f4_both_missing.shape

(287694, 29)

In [80]:
287694 + 57875 + 325907 + 328524

1000000

In [81]:
data.columns

Index(['index', 'vb_email', 'vb_tsmart_first_name', 'vb_tsmart_middle_name',
       'vb_tsmart_last_name', 'vb_voterbase_phone',
       'vb_voterbase_phone_wireless', 'vb_voterbase_dob',
       'coalesced_noncommercial_age', 'gender_noncommercial',
       'vb_tsmart_exact_track', 'vb_tsmart_full_address', 'vb_tsmart_city',
       'vb_tsmart_state', 'vb_tsmart_zip', 'vb_tsmart_zip4',
       'vb_tsmart_street_number', 'vb_tsmart_street_name',
       'vb_tsmart_street_suffix', 'vb_vf_reg_cass_address_full',
       'vb_vf_reg_cass_city', 'vb_vf_reg_cass_state', 'vb_vf_reg_cass_zip',
       'vb_vf_reg_cass_zip4', 'vb_vf_reg_cass_street_num',
       'vb_vf_reg_cass_street_name', 'vb_vf_reg_cass_street_suffix',
       'ts_reg_urbanicity_bins', 'full_name'],
      dtype='object')

In [87]:
def rename(df):
    
    df = df.rename(columns={
        "vb_tsmart_first_name" : "fn",
        "vb_tsmart_last_name" : "ln",
        "vb_voterbase_phone" : "phone",
        "vb_voterbase_dob" : "dob",
        "gender_noncommercial" : "gen",
        "vb_tsmart_city" : "ct",
        "vb_tsmart_state" : "st",
        "vb_tsmart_zip" : "zip",
        "vb_email" : "email"
    })
    df['age'] = 2020 - int(str(df.dob)[:4])
    df['country'] = "US"
    
    df = df[['fn', 'ln', 'phone', 'dob', 'age', 'gen', 'ct', 'st', 'zip', 'email', 'country']]
    
    return df

In [91]:
f1 = rename(f1_both_email_phone)

In [93]:
f2 = rename(f2_phone_only)

In [95]:
f3 = rename(f3_email_only)

In [96]:
f4 = rename(f4_both_missing)

In [99]:
f1.to_csv("f1_both_email_phone.csv")

In [100]:
f2.to_csv("f2_phone_only.csv")

In [101]:
f3.to_csv("f3_email_only.csv")

In [102]:
f4.to_csv("f4_both_missing.csv")

In [104]:
f1b = f1
f1b['email'] = np.nan
f1b['phone'] = np.nan
f1b.to_csv("f1b_both_removed.csv")

In [105]:
f2b = f2
f2b['phone'] = np.nan
f2b.to_csv("f2b_phone_removed.csv")

In [106]:
f3b = f3
f3b['email'] = np.nan
f3b.to_csv("f3b_email_removed.csv")

In [108]:
f1.shape[0] + f2.shape[0] + f3.shape[0] + f4.shape[0]

1000000